In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 🔍 Quick Version Comparison

# COMMAND ----------
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

client = MlflowClient()
model_name = "house_price_regression"

# Get all versions
versions = client.search_model_versions(f"name='{model_name}'")

print("="*80)
print(f"📊 COMPARING ALL VERSIONS OF '{model_name}'")
print("="*80)

# Compare each version
comparison_data = []

for v in sorted(versions, key=lambda x: int(x.version)):
    run = client.get_run(v.run_id)
    
    print(f"\n{'='*80}")
    print(f"📦 VERSION {v.version} - Stage: {v.current_stage}")
    print(f"{'='*80}")
    print(f"Created: {v.creation_timestamp}")
    print(f"Run ID: {v.run_id[:8]}")
    print(f"\n📊 Metrics:")
    print(f"   RMSE: ${run.data.metrics.get('test_rmse', 0):,.2f}")
    print(f"   MAE:  ${run.data.metrics.get('test_mae', 0):,.2f}")
    print(f"   R²:   {run.data.metrics.get('test_r2', 0):.4f}")
    print(f"\n⚙️  Parameters:")
    print(f"   Model Type: {run.data.params.get('model_type', 'N/A')}")
    print(f"   Random State: {run.data.params.get('random_state', 'N/A')}")
    
    comparison_data.append({
        'Version': v.version,
        'Stage': v.current_stage,
        'RMSE': run.data.metrics.get('test_rmse', 0),
        'MAE': run.data.metrics.get('test_mae', 0),
        'R²': run.data.metrics.get('test_r2', 0),
        'Random State': run.data.params.get('random_state', 'N/A')
    })

# Create comparison table
print(f"\n{'='*80}")
print("📊 SUMMARY TABLE")
print("="*80)

df = pd.DataFrame(comparison_data)
display(df)

# Find best
best_version = df.loc[df['R²'].idxmax()]

print(f"\n{'='*80}")
print(f"🏆 BEST VERSION: {best_version['Version']} (Highest R² = {best_version['R²']:.4f})")
print(f"   Current Stage: {best_version['Stage']}")
print("="*80)